# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

In this notebook, we'll perform a hermes energy calculation on a ligand derived from a smiles string

# 0) Complete example
See the [sample notebook](/Quickstarts/auto3d_conformer_tautomer_generation-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

## 1.1) Configuration

In [ ]:
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys

os.chdir(WORK_DIR)

## 1.2) Build your client

In [ ]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(batch_tags=TAGS)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(
    batch_tags=TAGS, restore_by_default=True
)

In [ ]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

# 2) Preparation
We want to convert our raw files into prepared qdxfs, with correct charges and missing residues filled

In [ ]:
help(client.auto3d)

Help on function auto3d in module rush.provider:

async auto3d(*args: *tuple[RushObject[bytes], str, Record], target: 'Target | None' = None, resources: 'Resources | None' = {'storage': 18, 'storage_units': 'MB', 'gpus': 1}, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[None], RushObject[list[Record]]]
    Generate 3D conformers from SMILES strings and other inputs

    Module version:
    `github:talo/tengu-auto3d/4896c5559f73046314ac4aaa0f660d86a0d259d7#auto3d_tengu`

    QDX Type Description:

        molecule_file: Object[@$Bytes];
        molecule_file_type: string;
        options: Auto3dOptions {
            mpi_np: u32?,
            patience: u32?,
            window: f32?,
            convergence_threshold: f32?,
            memory: u32?,
            enumerate_tautomer: bool?,
            max_confs: u32?,
            k: i32?,
            optimizing_engine: Auto3dOptimizingEngines[ANI2x | ANI2xt | AIMNET]?,
            job_name: string?,
  

In [ ]:
ligand_path = client.workspace / "aspirin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = await client.auto3d(
    ligand_path,
    "smi",
    {"k": 5, "use_gpu": True},
    resources={"gpus": 1, "storage": "5", "storage_units": "GB"},
    target="NIX_SSH_2",
)

print(f"{datetime.now().time()} | Running ligand preparation!")

2024-02-28 18:16:09,923 - rush - INFO - Trying to restore job with tags: ['qdx', 'tengu-py-auto3d_hermes', 'ASPRIN'] and path: github:talo/tengu-auto3d/4896c5559f73046314ac4aaa0f660d86a0d259d7#auto3d_tengu
18:16:10.061845 | Running ligand preparation!


In [ ]:
conformer_qdxf.source

UUID('c63db6e7-ed13-40a7-a241-16644869cfd5')

In [ ]:
try:
    await conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

2024-02-28 18:16:10,203 - rush - INFO - Argument efe2d408-1b15-4b68-ac70-0bc178f4880e is now ModuleInstanceStatus.RESOLVING
2024-02-28 18:16:13,542 - rush - INFO - Argument efe2d408-1b15-4b68-ac70-0bc178f4880e is now ModuleInstanceStatus.ADMITTED
2024-02-28 18:16:28,029 - rush - INFO - Argument efe2d408-1b15-4b68-ac70-0bc178f4880e is now ModuleInstanceStatus.DISPATCHED
2024-02-28 18:16:34,690 - rush - INFO - Argument efe2d408-1b15-4b68-ac70-0bc178f4880e is now ModuleInstanceStatus.RUNNING
2024-02-28 18:17:23,279 - rush - INFO - Argument efe2d408-1b15-4b68-ac70-0bc178f4880e is now ModuleInstanceStatus.AWAITING_UPLOAD


You should visualize your prepared ligand to spot check any issues

In [ ]:
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# 3) Quantum energy calculation
Finally, we submit our fragmented protein for quantum energy calculation, with custom configuration.

In [ ]:
help(client.hermes_energy)

Help on function hermes_energy in module rush.provider:

async hermes_energy(*args: *tuple[RushObject[Record], Record, Record | None, Record | None], target: 'Target | None' = None, resources: 'Resources | None' = {'storage': 1034, 'storage_units': 'MB', 'gpus': 4}, tags: 'list[str] | None' = None, restore: 'bool | None' = None) -> tuple[RushObject[Record]]
    Runs a HERMES energy calculation given a topology, and optionally model and keyword configurations.
    Will use the default model and keywords if none are provided

    Module version:
    `github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#hermes_energy`

    QDX Type Description:

        input: Object[Conformer];
        system: System {
            teams_per_node: u32?,
            max_gpu_memory_mb: u32?,
            oversubscribe_gpus: bool?,
            gpus_per_team: u32?
        };
        model: Model {
            aux_basis: string?,
            basis: string,
            force_cartesian_basis_sets: b

In [ ]:
HERMES_RESOURCES = {
    "gpus": 1,
    "storage": 100,
    "storage_units": "MB",
    "walltime": 60,
}

In [ ]:
(conformer,) = await client.pick_conformer(conformer_qdxf, 0)

2024-02-28 18:17:56,998 - rush - INFO - Trying to restore job with tags: ['qdx', 'tengu-py-auto3d_hermes', 'ASPRIN'] and path: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#pick_conformer


In [ ]:
conformer_out = json.load(open(await conformer.download(), "r"))

2024-02-28 18:17:57,235 - rush - INFO - Argument 600826f3-b93f-4080-9026-910a62cdb26f is now ModuleInstanceStatus.RESOLVING
2024-02-28 18:17:59,470 - rush - INFO - Argument 600826f3-b93f-4080-9026-910a62cdb26f is now ModuleInstanceStatus.ADMITTED
2024-02-28 18:18:10,829 - rush - INFO - Argument 600826f3-b93f-4080-9026-910a62cdb26f is now ModuleInstanceStatus.DISPATCHED
2024-02-28 18:18:16,401 - rush - INFO - Argument 600826f3-b93f-4080-9026-910a62cdb26f is now ModuleInstanceStatus.AWAITING_UPLOAD
2024-02-28 18:18:43,594 - rush - INFO - Argument 600826f3-b93f-4080-9026-910a62cdb26f is now ModuleInstanceStatus.COMPLETED


In [ ]:
(hermes_energy,) = await client.hermes_energy(
    conformer,
    {},
    {
        "method": "RestrictedRIMP2",
        "basis": "cc-pVDZ",
        "aux_basis": "cc-pVDZ-RIFIT",
        "force_cartesian_basis_sets": True,
        "standard_orientation": "FullSystem",
    },
    {
        "scf": {
            "max_iter": 50,
            "max_diis_history_length": 12,
            "convergence_metric": "DIIS",
            "convergence_threshold": 0.000001,
            "density_threshold": 1e-10,
            "density_basis_set_projection_fallback_enabled": True,
        },
        "frag": {
            "cutoffs": {"dimer": 22},
            "cutoff_type": "Centroid",
            "level": "Dimer",
            "reference_fragment": len(conformer_out["topology"]["fragments"]) - 1,
        },
    },
    target="NIX_SSH",
    resources=HERMES_RESOURCES,
)

2024-02-28 18:23:02,690 - rush - INFO - Trying to restore job with tags: ['qdx', 'tengu-py-auto3d_hermes', 'ASPRIN'] and path: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#hermes_energy


In [ ]:
await hermes_energy.get()

2024-02-28 18:23:03,019 - rush - INFO - Argument 891c3bce-df72-413d-9dee-c8d3a40b2186 is now ModuleInstanceStatus.RESOLVING
2024-02-28 18:23:06,413 - rush - INFO - Argument 891c3bce-df72-413d-9dee-c8d3a40b2186 is now ModuleInstanceStatus.ADMITTED
2024-02-28 18:23:16,545 - rush - INFO - Argument 891c3bce-df72-413d-9dee-c8d3a40b2186 is now ModuleInstanceStatus.DISPATCHED
2024-02-28 18:23:22,206 - rush - INFO - Argument 891c3bce-df72-413d-9dee-c8d3a40b2186 is now ModuleInstanceStatus.AWAITING_UPLOAD


ObjectUrlObject(url='https://storage.googleapis.com/rush_store_default/f6b507bd-d005-40e3-b2d9-af4ebc1c8d6f?x-goog-signature=47b2614a38c27af70559674c8ea5544c02db52aa35919485d88dc4858c72118b8ac66d0718645aded0c60978bdfd487c644fd413c5a5ab47e4c011af98396592c1fbf1e6357736772d07c4277a6efd41cd7e0c8ebf7b798dd9438f0a4f5cfc0eb43b38cd7f06f124f6504b5d9d004723bc63a69751b5460f31d63bfb7572a592ec20c2fbd8c4b889cd5b42ff63235c3dda590451817e38645a9e81af3526f9d1d291eb88169f56b645159f3af8ac3816884be746f74d0bb2809420736a02285b467917f75a541d551235c002ee8add17f76492f73f9817fd54b2cd1db334c8efb60bc7103b8b06fd5a85210c4752cf313dc5ed6b9242b6edd782fe23f38eadbc&x-goog-algorithm=GOOG4-RSA-SHA256&x-goog-credential=qdx-store-user%40humming-bird-321603.iam.gserviceaccount.com%2F20240228%2Fasia-southeast1%2Fstorage%2Fgoog4_request&x-goog-date=20240228T102351Z&x-goog-expires=3600&x-goog-signedheaders=host')